<a href="https://colab.research.google.com/github/lkp520/peo_seg/blob/main/%E2%80%9CPeople_segmentation1%E2%80%9D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%matplotlib inline

In [3]:
from pylab import imshow

In [4]:
import numpy as np

In [5]:
import os
import cv2

In [6]:
import torch

In [7]:
import albumentations as albu

In [8]:
!pip install iglovikov_helper_functions

In [9]:
from iglovikov_helper_functions.utils.image_utils import load_rgb, pad, unpad
from iglovikov_helper_functions.dl.pytorch.utils import tensor_from_rgb_image

In [10]:
# !wget https://habrastorage.org/webt/em/l7/cr/eml7crxnxftrimsmolwjegqcrp4.jpeg > /dev/null

In [11]:
!pip install tensorboard==2.5

     |████████████████████████████████| 6.0 MB 5.6 MB/s 
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytorch-lightning 1.3.8 requires tensorboard!=2.5.0,>=2.2.0, but you have tensorboard 2.5.0 which is incompatible.


In [12]:
!pip install people_segmentation  > /dev/null

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.5.0 requires tensorboard~=2.5, but you have tensorboard 2.4.1 which is incompatible.


In [13]:
from people_segmentation.pre_trained_models import create_model

In [14]:
model = create_model("Unet_2020-07-20")

/usr/local/lib/python3.7/dist-packages/torch/hub.py:480: UserWarning: Falling back to the old format < 1.6. This support will be deprecated in favor of default zipfile format introduced in 1.6. Please redo torch.save() to save it in the new zipfile format.
  warnings.warn('Falling back to the old format < 1.6. This support will be '


In [15]:
model.eval();

In [16]:
import os
import cv2
from PIL import Image
from google.colab import files 
import tarfile

def make_targz_one_by_one(output_filename, source_dir):
  tar = tarfile.open(output_filename,"w")
  for root,dir_name,files_list in os.walk(source_dir):
    for file in files_list:
      pathfile = os.path.join(root, file)
      tar.add(pathfile)
  tar.close()
  files.download(output_filename)

In [1]:
path = 'ds7/'
mask_path = 'mask7/'
path_list = os.listdir(path)

for i in path_list:
  print(i)
  image = load_rgb(path+i)
  transform = albu.Compose([albu.Normalize(p=1)], p=1)
  padded_image, pads = pad(image, factor=32, border=cv2.BORDER_CONSTANT)
  x = transform(image=padded_image)["image"]
  x = torch.unsqueeze(tensor_from_rgb_image(x), 0)
  with torch.no_grad():
    prediction = model(x)[0][0]
  mask = (prediction > 0).cpu().numpy().astype(np.uint8)
  mask = unpad(mask, pads)
  cv2.imwrite(mask_path+i, mask * 255)
  # dst = cv2.addWeighted(image, 1, (cv2.cvtColor(mask, cv2.COLOR_GRAY2RGB) * (0, 255, 0)).astype(np.uint8), 0.5, 0)
  # img = Image.fromarray(np.uint8(dst))
  # img.save(result_path+i)

make_targz_one_by_one('maskds7', 'mask7')

NameError: ignored

In [40]:
!rm -rf ds8/
!rm -rf mask8/

In [41]:
import glob
path_file_number = glob.glob('ds7/*.jpeg') # 指定文件夹下文件个数
print(len(path_file_number))

232


In [32]:
!mkdir /content/ds8
!mkdir /content/mask8

In [ ]:
!pip clone https://github.com/supervisely/supervisely

In [ ]:
# 生成mask(label)

import os
import numpy as np
import cv2
import tqdm
import supervisely_lib as sly
from matplotlib import pyplot as plt
 
def genMask(dataset_root_dir="/ds7",
            output_dir="/mask",
            label_color=[255,255,255]):
    ''' 生成Mask
        dataset_root_dir: 数据集根目录
        output_dir: 生成mask的保存目录
        label_color: mask 标注的颜色[R, G, B]
    '''
    project = sly.Project(dataset_root_dir, sly.OpenMode.READ)
 
    # 打印数据集信息
    print(f"Project name     : {project.name}")
    print(f"Project directory: {project.directory}")
    print(f"Total images     : {project.total_items}")
    print(f"Dataset names    : {project.datasets.keys()}")
    print()
    print(project.meta)
 
    pbar = tqdm.tqdm(total=project.total_items)
    for dataset in project:
        for item_name in dataset:
            # 更新进度条
            pbar.update(1)
            # 获取原始图像和标注文件路径
            item_paths = dataset.get_item_paths(item_name)
            # 加载注释文件
            ann = sly.Annotation.load_json_file(item_paths.ann_path, project.meta)
            # 创建一个用于渲染标注的3通道黑色画布，
            ann_render = np.zeros(ann.img_size + (3,), dtype=np.uint8)
            # 渲染所有的标注（该数据集只有人）
            ann.draw(ann_render, color=label_color)
            # ann_render shape: (h, w, c), pixel: (R, G, B)
            # RGB -> BGR，用于Opencv
            ann_render = ann_render[..., ::-1]
            # mask 保存目录不存在，则创建
            save_dir = os.path.join(output_dir, dataset.name)
            if not os.path.exists(save_dir):
                os.makedirs(save_dir)
            # 保存mask
            mask_path = os.path.join(save_dir, item_name)
            cv2.imwrite(mask_path, ann_render)
    pbar.close()

if __name__ == '__main__':
    import fire
    fire.Fire(genMask, name='gen_mask')